In [ ]:
import pandas as pd
import numpy as np
import string
from string import digits
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
import tensorflow as tf
import os


In [ ]:
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
data=pd.read_table('../input/languages/mar.txt',names=['lang','trans','useless'])
data.drop('useless',axis=1,inplace=True)
# data=data[:100000]
spchrs=set(string.punctuation)
lowerCase=lambda x:x.lower()
quotes=lambda x:re.sub("'",'',x)
specialCh=lambda x:''.join(ch for ch in x if ch not in spchrs)
rmDigits=lambda x:x.translate(str.maketrans('','',digits))
spaces=lambda x:x.strip()
unwanted=lambda x:re.sub(" +"," ",x)
startEnd=lambda x:'START_ '+x+' _END'
data.lang=data.lang.apply(lowerCase)
data.lang=data.lang.apply(quotes)
data.lang=data.lang.apply(specialCh)
data.lang=data.lang.apply(rmDigits)
data.lang=data.lang.apply(spaces)
data.lang=data.lang.apply(unwanted)
data.trans=data.trans.apply(lowerCase)
data.trans=data.trans.apply(quotes)
data.trans=data.trans.apply(specialCh)
data.trans=data.trans.apply(rmDigits)
data.trans=data.trans.apply(spaces)
data.trans=data.trans.apply(unwanted)
data.trans=data.trans.apply(startEnd)

In [ ]:
langVocab=set()
for line in data.lang:
    for word in line.split():
        langVocab.add(word)
transVocab=set()
for line in data.trans:
    for word in line.split():
        transVocab.add(word)
maxSrcLen=0
for line in data.lang:
    maxSrcLen=max(maxSrcLen,len(line.split()))
print(maxSrcLen)
maxTarLen=0
for line in data.trans:
    maxTarLen=max(maxTarLen,len(line.split()))
print(maxTarLen)

In [ ]:
inputWords=sorted(list(langVocab))
targetWords=sorted(list(transVocab))
lenOfEncoderTokens=len(langVocab)
lenOfDecoderTokens=len(transVocab)
print(lenOfDecoderTokens,lenOfEncoderTokens)
lenOfDecoderTokens+=1
lenOfDecoderTokens
tarTokenInd,inpRevIndMap,tarRevIndMap,inpTokenInd={},{},{},{}
for i,word in enumerate(inputWords):
    inpTokenInd[word]=i+1
    inpRevIndMap[i]=word
for i,word in enumerate(targetWords):
    tarTokenInd[word]=i+1
    tarRevIndMap[i]=word

In [ ]:
x,y=data.lang,data.trans
x_train,x_test,y_train,y_test=train=train_test_split(x,y,test_size=0.1)
def encode(x=x_train,y=y_train,size=128):
    while True:
        for i1 in range(0,len(x),size):
            encInp=np.zeros((size,maxSrcLen),dtype='float32')
            decInp=np.zeros((size,maxTarLen),dtype='float32')
            decTar=np.zeros((size,maxTarLen,lenOfDecoderTokens),dtype='float32')
            for i2,(inpText,tarText) in enumerate(zip(x[i1:i1+size],y[i1:i1+size])):
                for i3,word in enumerate(inpText.split()):
                    encInp[i2,i3]=inpTokenInd[word]
                tarTextSplit=tarText.split()
                for i3,word in enumerate(tarTextSplit):
                    if i3<len(tarTextSplit)-1:
                        decInp[i2,i3]=tarTokenInd[word]=tarTokenInd[word]
                    if i3>0:
                        decTar[i2,i3-1,tarTokenInd[word]]=1
            yield([encInp,decInp],decTar)
dims=50
encInp=Input(shape=(None,))
encEmb=Embedding(lenOfEncoderTokens,dims,mask_zero=True)(encInp)
encLSTM=LSTM(dims,return_state=True)
encOut,stateH,stateC=encLSTM(encEmb)
encStates=[stateH,stateC]
decInp=Input(shape=(None,))
decEmbLayer=Embedding(lenOfDecoderTokens,dims,mask_zero=True)
decEmb=decEmbLayer(decInp)
decLSTM=LSTM(dims,return_sequences=True,return_state=True)
decOut,_,_=decLSTM(decEmb,initial_state=encStates)
decDense=Dense(lenOfDecoderTokens,activation='softmax')
decOut=decDense(decOut)
model=Model([encInp,decInp],decOut)
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['acc'])

In [ ]:
print(lenOfEncoderTokens,dims,lenOfDecoderTokens,maxSrcLen)
dims=50

In [ ]:
model.load_weights('../input/mar-weight/mar_weights.h5')

In [ ]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encInp,encStates)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(dims,))
decoder_state_input_c = Input(shape=(dims,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= decEmbLayer(decInp) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decLSTM(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decDense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decInp] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = tarTokenInd['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = tarRevIndMap[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence



In [ ]:
train_gen = encode(x_train, y_train, size = 1)
k=-1


In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', x_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])